In [22]:
%run ~/glre/relmapping/annot_cb/notebooks/annot__init__.ipynb

os.getcwd(): /mnt/beegfs/home1/ahringer/fnc21/glre/relmapping


In [23]:
def mp(fp): return os.path.join('annot_cb', 'metrics_exon', fp)

fp_atac = 'annot_cb/accessible_sites_cb.tsv'
df_atac = pd.read_csv(fp_atac, sep='\t')
l_atac_peak_pos = df_atac[['start', 'end']].mean(axis=1).map(int)
df_atac.head()

,chrom,concave_start,concave_end,mode,wt_ya_rep1_score,wt_ya_rep2_score,wt_ya_prp1_score,wt_ya_prp2_score,glp1_ya_rep1_score,glp1_ya_rep2_score,glp1_ya_prp1_score,glp1_ya_prp2_score,min_rank,start,end,wt_ya_globalIDR,glp1_ya_globalIDR,max_globalIDR
0,I,2947,3135,3049,40.389435,77.209199,63.212697,57.973379,63.650777,120.725868,112.125014,58.964616,6274.0,2974,3125,2.141424,0.826997,2.141424
1,I,7979,8179,8076,139.952168,252.490442,224.079170,179.322426,145.487546,136.277249,113.510888,170.404159,3712.0,8001,8152,5.000000,4.292369,5.000000
2,I,16604,16805,16689,1057.104227,1091.101451,1056.279501,1095.246373,169.335348,160.960105,166.327516,165.916485,1188.0,16614,16765,5.000000,5.000000,5.000000
3,I,17075,17277,17167,659.079464,636.991886,650.436304,643.481255,42.036328,24.001806,25.143059,45.093767,2385.0,17092,17243,5.000000,0.245042,5.000000
4,I,21471,21680,21563,3208.708694,2859.586826,2933.962826,3100.339499,93.515794,170.015699,140.077836,105.649899,58.0,21488,21639,5.000000,3.895870,5.000000


In [57]:
# df_exon1 / df_exon2: first/other exons, coordinates fixed to represent 5' ends
#NAMES_EXON = ('chrom', 'start', 'end', 'transcript_id', 'score', 'strand', 'gene_id', 'gene_biotype', 'display_id')
l_adj_ = [
    'chrom', 'start', 'end', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'Name', 
    'exon_start', 'exon_end', 'utr_start', 'utr_end', 'aoe_end', 'aoe_start', 'gene_biotype', 
    'exon_id', 'gene_id', 'init_cutoff_pos', #'locus_id', 
]

df_pass1_exon1_fwd = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass1_exon1_fwd_utradj.bed')[l_adj_]
df_pass1_exon1_rev = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass1_exon1_rev_utradj.bed')[l_adj_]
df_pass1_exon2_fwd = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass1_exon2_fwd.bed')
df_pass1_exon2_rev = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass1_exon2_rev.bed')
df_pass2_exon1_fwd = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass2_exon1_fwd.bed')
df_pass2_exon1_rev = read_gffbed('annot_cb/canonical_geneset/WS275_CB4.transcripts.annot_pass2_exon1_rev.bed')

df_pass1_exon1_fwd["utr_end"] = pd.to_numeric(df_pass1_exon1_fwd["utr_end"], errors='coerce')
df_pass1_exon1_fwd["utr_end"] = df_pass1_exon1_fwd["utr_end"].replace(np.nan, 0)
df_pass1_exon1_fwd["utr_end"] = df_pass1_exon1_fwd["utr_end"].astype(int)
#df_pass1_exon1_fwd["utr_end"] = df_pass1_exon1_fwd["utr_end"].astype(str).astype(float)
#df_pass1_exon1_fwd["start"] = df_pass1_exon1_fwd["start"].astype(str).astype(float)
#df_pass1_exon1_fwd["end"] = df_pass1_exon1_fwd["end"].astype(str).astype(float)

df_pass1_exon1_rev["utr_start"] = pd.to_numeric(df_pass1_exon1_rev["utr_start"], errors='coerce')
df_pass1_exon1_rev["utr_start"] = df_pass1_exon1_rev["utr_start"].replace(np.nan, 0)
df_pass1_exon1_rev["utr_start"] = df_pass1_exon1_rev["utr_start"].astype(int)
#df_pass1_exon1_rev["utr_start"] = df_pass1_exon1_rev["utr_start"].astype(str).astype(float)
#df_pass1_exon1_rev["start"] = df_pass1_exon1_rev["start"].astype(str).astype(float)
#df_pass1_exon1_rev["end"] = df_pass1_exon1_rev["end"].astype(str).astype(float)

df_pass1_exon1_fwd.loc[(df_pass1_exon1_fwd['utr_end'] > df_pass1_exon1_fwd['start'] + 251) & (df_pass1_exon1_fwd['utr_end'] - df_pass1_exon1_fwd['start'] < 1000), 'end'] = df_pass1_exon1_fwd['utr_end']
df_pass1_exon1_fwd.loc[(df_pass1_exon1_fwd['utr_end'] < df_pass1_exon1_fwd['start'] + 251) | (df_pass1_exon1_fwd['utr_end'] - df_pass1_exon1_fwd['start'] > 1000), 'end'] = df_pass1_exon1_fwd['start'] + 251

df_pass1_exon1_rev.loc[(df_pass1_exon1_rev['utr_start'] > df_pass1_exon1_rev['end'] - 251) & (df_pass1_exon1_rev['utr_start'] - df_pass1_exon1_rev['end'] < 1000), 'start'] = df_pass1_exon1_rev['utr_start']
df_pass1_exon1_rev.loc[(df_pass1_exon1_rev['utr_start'] < df_pass1_exon1_rev['end'] - 251) | (df_pass1_exon1_rev['utr_start'] - df_pass1_exon1_rev['end'] > 1000), 'start'] = df_pass1_exon1_rev['end'] - 251


# Set 1st exon matching area to TSS + 250bp
#df_pass1_exon1_fwd['end'] = df_pass1_exon1_fwd['start'] + 1 + 250
#df_pass1_exon1_rev['start'] = df_pass1_exon1_rev['end'] - 1 - 250

print(len(df_pass1_exon1_fwd), len(df_pass1_exon1_rev),
      len(df_pass1_exon2_fwd), len(df_pass1_exon2_rev),
      len(df_pass2_exon1_fwd), len(df_pass2_exon1_rev),
)

12641 12384 69976 67918 478 496


In [58]:
# Match hypersensitive sites to closest first/other exon
def df_closest(df_a, df_b, b_prefix, **kwargs):
    df_a_sort = df_a
    df_b_sort = df_b.sort_values(list(df_b.columns[:3]))
    fn_ = BedTool.from_dataframe(df_a).closest(BedTool.from_dataframe(df_b_sort).fn, **kwargs).fn
    names_ = list(itertools.chain(df_a.columns.values,
        ['%s_%s' % (b_prefix, col) for col in df_b.columns.values],
        ['%s_distance' % (b_prefix)]
    ))
    df_ = pd.read_csv(fn_, names=names_, sep='\t')
    return df_[names_[len(df_a.columns):]]

# Closest exon1 -- protein_coding & pseudogene
df_ = pd.concat([df_atac['chrom'], l_atac_peak_pos, l_atac_peak_pos + 1], axis=1).copy()
df_pass1_exon1_fwd_out_ = df_closest(df_, df_pass1_exon1_fwd, 'pass1_exon1', D='ref', t='first', iu=True)
df_pass1_exon1_rev_out_ = df_closest(df_, df_pass1_exon1_rev, 'pass1_exon1', D='ref', t='last', id=True)

# Closest exon2 can be both upstream or downstream
df_pass1_exon2_fwd_out_ = df_closest(df_, df_pass1_exon2_fwd, 'pass1_exon2', D='ref', t='first')
df_pass1_exon2_rev_out_ = df_closest(df_, df_pass1_exon2_rev, 'pass1_exon2', D='ref', t='last')

# Closest exon2 can be both upstream or downstream (downstream only)
df_pass1_exon2_downstream_fwd_out_ = df_closest(df_, df_pass1_exon2_fwd, 'pass1_exon2_downstream', D='ref', t='first', iu=True)
df_pass1_exon2_downstream_rev_out_ = df_closest(df_, df_pass1_exon2_rev, 'pass1_exon2_downstream', D='ref', t='last', id=True)

# Closes pass2/exon1 -- tRNA, snoRNA, rRNA, miRNA, etc
df_pass2_exon1_fwd_out_ = df_closest(df_, df_pass2_exon1_fwd, 'pass2_exon1', D='ref', t='first', iu=True)
df_pass2_exon1_rev_out_ = df_closest(df_, df_pass2_exon1_rev, 'pass2_exon1', D='ref', t='last', id=True)

df_exon_fwd = pd.concat([df_pass1_exon1_fwd_out_, df_pass1_exon2_fwd_out_, df_pass1_exon2_downstream_fwd_out_, df_pass2_exon1_fwd_out_], axis=1)
df_exon_rev = pd.concat([df_pass1_exon1_rev_out_, df_pass1_exon2_rev_out_, df_pass1_exon2_downstream_rev_out_, df_pass2_exon1_rev_out_], axis=1)

df_exon_fwd.to_csv(mp('closest_exon_fwd.tsv'), header=True, index=False, sep='\t')
df_exon_rev.to_csv(mp('closest_exon_rev.tsv'), header=True, index=False, sep='\t')

#l_ = ['pass1_exon1_gene_id', 'pass1_exon1_locus_id', 'pass1_exon1_distance']
l_ = ['pass1_exon1_gene_id', 'pass1_exon1_distance']
write_gffbed(mp('closest_pass1_exon1_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon1_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon1_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon1_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass1_exon2_gene_id', 'pass1_exon2_locus_id', 'pass1_exon2_distance']
l_ = ['pass1_exon2_gene_id', 'pass1_exon2_distance']
write_gffbed(mp('closest_pass1_exon2_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon2_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon2_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon2_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass1_exon2_downstream_gene_id', 'pass1_exon2_downstream_locus_id', 'pass1_exon2_downstream_distance']
l_ = ['pass1_exon2_downstream_gene_id', 'pass1_exon2_downstream_distance']
write_gffbed(mp('closest_pass1_exon2_downstream_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon2_downstream_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon2_downstream_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon2_downstream_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass2_exon1_gene_id', 'pass2_exon1_locus_id', 'pass2_exon1_distance']
l_ = ['pass2_exon1_gene_id', 'pass2_exon1_distance']
write_gffbed(mp('closest_pass2_exon1_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass2_exon1_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass2_exon1_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass2_exon1_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)